# General Instructions to students:

1. There are 5 types of cells in this notebook. The cell type will be indicated within the cell.
    1. Markdown cells with problem written in it. (DO NOT TOUCH THESE CELLS) (**Cell type: TextRead**)
    2. Python cells with setup code for further evaluations. (DO NOT TOUCH THESE CELLS) (**Cell type: CodeRead**)
    3. Python code cells with some template code or empty cell. (FILL CODE IN THESE CELLS BASED ON INSTRUCTIONS IN CURRENT AND PREVIOUS CELLS) (**Cell type: CodeWrite**)
    4. Markdown cells where a written reasoning or conclusion is expected. (WRITE SENTENCES IN THESE CELLS) (**Cell type: TextWrite**)
    5. Temporary code cells for convenience and TAs. (YOU MAY DO WHAT YOU WILL WITH THESE CELLS, TAs WILL REPLACE WHATEVER YOU WRITE HERE WITH OFFICIAL EVALUATION CODE) (**Cell type: Convenience**)
    
2. You are not allowed to insert new cells in the submitted notebook.

3. You are not allowed to import any extra packages.

4. The code is to be written in Python 3.6 syntax. Latest versions of other packages maybe assumed.

5. In CodeWrite Cells, the only outputs to be given are plots asked in the question. Nothing else to be output/print. 

6. If TextWrite cells ask you to give accuracy/error/other numbers you can print them on the code cells, but remove the print statements before submitting.

7. The convenience code can be used to check the expected syntax of the functions. At a minimum, your entire notebook must run with "run all" with the convenience cells as it is. Any runtime failures on the submitted notebook as it is will get zero marks.

8. All code must be written by yourself. Copying from other students/material on the web is strictly prohibited. Any violations will result in zero marks.

9. All datasets will be given as .npz files, and will contain data in 4 numpy arrays :"X_train, Y_train, X_test, Y_test". In that order. The meaning of the 4 arrays can be easily inferred from their names.

10. All plots must be labelled properly, all tables must have rows and columns named properly.

11. Before subbmission ensure that you submit with the outputs (do not clear the outputs), so that when evaluating we can run selectively.

12. Before submission ensure that the path for the folder containing the data is "../../Data/" 


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import matplotlib as mpl
import pandas as pd7
import warnings
warnings.filterwarnings('ignore')

# 1. Logistic Regression 

Write code for doing logistic regression below. Also write code for choosing best hyperparameters for each kernel type (use a part of training set as validation set). 

The range of hyperparameters is typically chosen on a log scale e.g. 1e-4, 1e-3, 1e-2... 1e3.

Write code for running in the cell after (You may be asked to demonstrate your code during the viva using this cell.)

In text cell after that report the following numbers you get by running appropriate code:

For each classification data set report the best kernel and regularisation parameters for linear, RBF and Poly kernels. (Linear has no kernel parameter.) Report the training and test zero-one error for those hyperparameters. 

For each given hyperparameter setting (kernel and regularisation) you will have to do some exploring to find the right learning rate to use in gradient descent. The optimisation learning rate is not a model hyperparameter and hence can be chosen based on just the training set. i.e. choose the learning rate for which the training loss decreases the most.

For the synthetic classification datasets (dataset_A and dataset_B) in 2-dimensions, also illustrate the learned classifier for each kernel setting. Do this in the last codeWrite cell for this question.

In [3]:
# CodeWrite 
#Write logistic regression code from scratch. Use gradient descent.
# Only write functions here

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def compute_kernel_matrix(X, kernel, kernel_param):
    """
    Return the kernel matrix for the given kernel
    """
    if kernel == 'linear':
        return X @ X.T
    elif kernel == 'poly':
        return (1 + (X @ X.T))**kernel_param
    elif kernel == 'rbf':
        K = np.zeros((X.shape[0], X.shape[0]))
        for i in range(X.shape[0]):
            for j in range(X.shape[0]):
                K[i, j] = np.exp(-kernel_param * np.linalg.norm(X[i] - X[j])**2)
        return K

def train_pred_logistic_regression(X, Y, kernel='linear', reg_param=0., 
                                   kernel_param=1., num_iter_gd=100):
    r"""
    Arguments:
    X : (n,d) shape numpy array
    Y : (n,)  shape numpy array
    X_test : (m,d) shape numpy array
    kernel = 'linear' or 'rbf' or 'poly' 
    reg_param = $\lambda$
    num_iter_gd = number of GD iterations.

    Returns the result of kernel logistic regression :
    alpha: Vector of solutions for the dual. Numpy array of shape (n,)

    Primal problem:
    $ \min_w  \sum_{i=1}^n \log(1+\exp(-y_i* \w^\top \phi(\x_i)))  + \frac{\lambda}{2} ||\w||^2 $

    the dual of which is

    $ \min_alpha \sum_{i=1}^n \log(1+\exp(-y_i* \alpha^\top K_{:,i} ))  + \frac{\lambda}{2} \alpha^\top K \alpha $
    where $\phi$ is the feature got by the kernel.

    Where K is the nxn kernel matrix computed on the training data.

    The kernel is defined by the kernel_param:
    If kernel=linear: K(\u,\v) = \u^\top \v  
    If kernel=poly:  K(\u,\v) = (1+\u^\top \v)^(kernel_param)
    If kernel=rbf:  K(\u,\v) = \exp(-kernel_param*||\u-\v||^2)
    """
    mu = np.mean(X, axis=0)
    X = X - mu
    K = compute_kernel_matrix(X, kernel, kernel_param)

    alpha = np.zeros(X.shape[0])
    eta = 0.01

    for _ in range(num_iter_gd):
        grad = reg_param * (K @ alpha) + np.sum((1 - sigmoid(Y * (K @ alpha))) * (-Y * K), axis=1)
        alpha -= eta * grad
    return alpha

def test_pred(alpha, X, kernel, kernel_param):
    """
    Return the predictions on test_X using the learnt alphas
    """
    K = compute_kernel_matrix(X, kernel, kernel_param)
    return np.sign(K @ alpha)
    

In [16]:
# CodeWrite : Use the functions above to do validation to get best hyperparameters 
# (i.e. kernel_param and regularisation_param).
# Also, get the numbers you report below. 

kernels = ['linear', 'poly', 'rbf']
reg_params = [0.01, 0.1, 1, 10, 100]
kernel_params = [0.01, 0.1, 1, 10, 100]

def get_best_hyperparameters(file):
    data = np.load("./dataset/"+file)
    X_train = data['arr_0']
    Y_train = data['arr_1']
    X_test = data['arr_2']
    Y_test = data['arr_3']

    # 80 - 20 split
    x_train, x_val = X_train[:int(0.8 * X_train.shape[0])], X_train[int(0.2 * X_train.shape[0]):]
    y_train, y_val = Y_train[:int(0.8 * Y_train.shape[0])], Y_train[int(0.2 * Y_train.shape[0]):]

    best_kernel = None
    best_reg_param = None
    best_kernel_param = None
    best_accuracy = 0

    for kernel in kernels:
        for reg_param in reg_params:
            for kernel_param in kernel_params:
                alpha = train_pred_logistic_regression(x_train, y_train, kernel, reg_param, kernel_param, 100)
                y_pred = test_pred(alpha, x_val, kernel, kernel_param)
                accuracy = np.mean(y_pred == y_val)
                if accuracy > best_accuracy:
                    best_kernel = kernel
                    best_reg_param = reg_param
                    best_kernel_param = kernel_param
                    best_accuracy = accuracy

    print("Best kernel: ", best_kernel)
    print("Best reg_param: ", best_reg_param)
    print("Best kernel_param: ", best_kernel_param)
    print("Best accuracy: ", best_accuracy)

get_best_hyperparameters("dataset_A.npz")
get_best_hyperparameters("dataset_B.npz")
get_best_hyperparameters("dataset_C.npz")
get_best_hyperparameters("dataset_D.npz")


C:\Users\djhem\AppData\Local\Temp\ipykernel_2168\85186831.py:6: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
C:\Users\djhem\AppData\Local\Temp\ipykernel_2168\85186831.py:15: RuntimeWarning: invalid value encountered in power
  return (1 + np.matmul(X, X.T))**kernel_param
C:\Users\djhem\AppData\Local\Temp\ipykernel_2168\85186831.py:60: RuntimeWarning: overflow encountered in matmul
  grad = reg_param * (K @ alpha) - (1 - sigmoid(Y * (K @ alpha))) * (-Y @ K)
C:\Users\djhem\AppData\Local\Temp\ipykernel_2168\85186831.py:61: RuntimeWarning: invalid value encountered in subtract
  alpha -= eta * grad
C:\Users\djhem\AppData\Local\Temp\ipykernel_2168\85186831.py:60: RuntimeWarning: invalid value encountered in matmul
  grad = reg_param * (K @ alpha) - (1 - sigmoid(Y * (K @ alpha))) * (-Y @ K)
C:\Users\djhem\AppData\Local\Temp\ipykernel_2168\85186831.py:60: RuntimeWarning: overflow encountered in multiply
  grad = reg_param * (K @ alpha) - (1 - sigmoid(Y * (K @ alph

Best kernel:  poly
Best reg_param:  1
Best kernel_param:  1
Best accuracy:  0.5158333333333334
Best kernel:  linear
Best reg_param:  0.1
Best kernel_param:  0.01
Best accuracy:  0.6908333333333333


C:\Users\djhem\AppData\Local\Temp\ipykernel_2168\85186831.py:15: RuntimeWarning: overflow encountered in power
  return (1 + np.matmul(X, X.T))**kernel_param


Best kernel:  linear
Best reg_param:  0.01
Best kernel_param:  0.01
Best accuracy:  0.7133333333333334
Best kernel:  rbf
Best reg_param:  100
Best kernel_param:  0.01
Best accuracy:  0.65625


TextWrite Cell: Give your observations and the list of hyperparameter choices and train zero-one error  and test zero-one error for all three kernel choices, for all 4 datasets (2 real world and 2 synthetic).  




In [ ]:
# Codewrite cell: Generate plots of learned classifier for all three kernel types, on dataset_A and datasset_B.
# Plots should give both the learned classifier and the train data. 
# Similar to  Bishop Figure 4.5 (with just two classes here.)
# Total number of plots = 3 * 2 = 6



# 2. SVM

Write code for learning SVM below. Also write code for choosing best hyperparameters for each kernel type. You may use sklearn.svm for this purpose. (use a part of training set as validation set)

Write code for running in the cell after (You may be asked to demonstrate your code during the viva using this cell.)

In text cell after that report the following numbers you get by running appropriate code:

For each classification data set report the best kernel and regularisation parameters for linear, RBF and Poly kernels. (Linear has no kernel parameter.) Report the training and test zero-one error for those hyperparameters.

For the synthetic classification datasets in 2-dimensions, also illustrate the learned classifier for each kernel setting. Do this in the last codeWrite cell for this question.

In [4]:
# CodeWrite cell
# Write SVM classifier using SKlearn
# write only functions here

def train_pred_svm_sklearn(X, Y, kernel='linear', reg_param=1., kernel_param=1.):
    """
    Arguments:
    X : (n,d) shape numpy array
    Y : (n,)  shape numpy array
    kernel = 'linear' or 'rbf' or 'poly' 
    reg_param = C

    Returns the result of kernel logistic regression :
    clf: SVM classifier object
    """
    clf = svm.SVC(kernel=kernel, C=reg_param, coef0=1)
    if kernel == 'rbf':
        clf.set_params(**{'gamma': kernel_param})
    elif kernel == 'poly':
        clf.set_params(**{'degree': kernel_param, 'gamma': 'auto'})
    clf.fit(X, Y)
    return clf

def test_pred_svm_sklearn(clf, X):
    """
    Return the predictions on X using the learnt clf
    """
    return clf.predict(X)


In [5]:
# CodeWrite cell
# Write code here for doing validation (for kernel_param and regularisation_param)
# on a subset of the training set. 
# Also for generating the numbers that you report below.

kernels = ['linear', 'poly', 'rbf']

best_hyperparameters = {}

def get_best_hyperparameters_svm(file):
    print("Dataset: ", file)
    data = np.load("./dataset/"+file)

    X_train = data['arr_0']
    Y_train = data['arr_1']
    x_test = data['arr_2']
    y_test = data['arr_3']

    # 80 - 20 split
    x_train, x_val = X_train[:int(0.8 * X_train.shape[0])], X_train[int(0.2 * X_train.shape[0]):]
    y_train, y_val = Y_train[:int(0.8 * Y_train.shape[0])], Y_train[int(0.2 * Y_train.shape[0]):]

    best_params = {'linear': {'reg_param': 0, 'kernel_param': 0, 'accuracy': 0},
            'poly': {'reg_param': 0, 'kernel_param': 0, 'accuracy': 0},
            'rbf': {'reg_param': 0, 'kernel_param': 0, 'accuracy': 0}}

    for kernel in kernels:
        print("Kernel: ", kernel)
        if kernel == 'linear':
            reg_params = [10 ** i for i in range(-3, 3)]
            kernel_params = [1]
        elif kernel == 'poly':
            reg_params = [10 ** i for i in range(-3, 1)]
            kernel_params = [2 ** i for i in range(4)]
        elif kernel == 'rbf':
            reg_params = [10 ** i for i in range(-3, 3)]
            kernel_params = [10 ** i for i in range(-3, 3)]

        best_reg_param = None
        best_kernel_param = None
        best_val_accuracy = 0
        
        for reg_param in reg_params:
            for kernel_param in kernel_params:
                clf = train_pred_svm_sklearn(x_train, y_train, kernel, reg_param, kernel_param)
                y_pred = test_pred_svm_sklearn(clf, x_val)
                accuracy = np.mean(y_pred == y_val)
                if accuracy > best_val_accuracy:
                    best_reg_param = reg_param
                    best_kernel_param = kernel_param
                    best_val_accuracy = accuracy

        print("Best reg_param: ", best_reg_param)
        print("Best kernel_param: ", best_kernel_param)
        print("Best accuracy: ", best_val_accuracy)

        clf = train_pred_svm_sklearn(x_train, y_train, kernel, best_reg_param, best_kernel_param)
        y_pred = test_pred_svm_sklearn(clf, x_test)
        test_accuracy = np.mean(y_pred == y_test)
        print("Test accuracy: ", test_accuracy)

        best_params[kernel]['reg_param'] = best_reg_param
        best_params[kernel]['kernel_param'] = best_kernel_param
        best_params[kernel]['accuracy'] = test_accuracy

    best_hyperparameters[file] = best_params

get_best_hyperparameters_svm("dataset_A.npz")
get_best_hyperparameters_svm("dataset_B.npz")
get_best_hyperparameters_svm("dataset_C.npz")
get_best_hyperparameters_svm("dataset_D.npz")

Dataset:  dataset_A.npz
Kernel:  linear
Best reg_param:  1
Best kernel_param:  1
Best accuracy:  0.8675
Test accuracy:  0.872
Kernel:  poly
Best reg_param:  0.1
Best kernel_param:  8
Best accuracy:  1.0
Test accuracy:  0.996
Kernel:  rbf
Best reg_param:  0.1
Best kernel_param:  100
Best accuracy:  1.0
Test accuracy:  0.998
Dataset:  dataset_B.npz
Kernel:  linear
Best reg_param:  1
Best kernel_param:  1
Best accuracy:  0.8291666666666667
Test accuracy:  0.804
Kernel:  poly
Best reg_param:  1
Best kernel_param:  1
Best accuracy:  0.8291666666666667
Test accuracy:  0.804
Kernel:  rbf
Best reg_param:  100
Best kernel_param:  100
Best accuracy:  0.9066666666666666
Test accuracy:  0.722
Dataset:  dataset_C.npz
Kernel:  linear
Best reg_param:  100
Best kernel_param:  1
Best accuracy:  0.9325
Test accuracy:  0.8956228956228957
Kernel:  poly
Best reg_param:  0.001
Best kernel_param:  4
Best accuracy:  0.9975
Test accuracy:  0.9696969696969697
Kernel:  rbf
Best reg_param:  10
Best kernel_param: 

TextWrite Cell: Give your observations and the list of hyperparameter choices and train zero-one error  and test zero-one error for all three kernel choices, for all 4 datasets (2 real world and 2 synthetic).  


In [ ]:
# Codewrite cell: Generate plots of learned classifier for all three kernel types, on dataset_A and datasset_B.
# Plots should give both the learned classifier and the train data. 
# Similar to  Bishop Figure 4.5 (with just two classes here.)
# Total number of plots = 3 * 2 = 6

def plot_decision_boundary(clf, x_train, y_train):
    x_min, x_max = x_train[:, 0].min() - 1, x_train[:, 0].max() + 1
    y_min, y_max = x_train[:, 1].min() - 1, x_train[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(x_train[:, 0], x_train[:, 1], c=y_train, s=20, edgecolor='k')

def plot_svm_sklearn(file):
    data = np.load("./dataset/"+file)

    x_train = data['arr_0']
    y_train = data['arr_1']

    for kernel in kernels:
        reg_param = best_hyperparameters[file][kernel]['reg_param']
        kernel_param = best_hyperparameters[file][kernel]['kernel_param']
        clf = train_pred_svm_sklearn(x_train, y_train, kernel, reg_param, kernel_param)
        plot_decision_boundary(clf, x_train, y_train)

plot_svm_sklearn("dataset_A.npz")
plot_svm_sklearn("dataset_B.npz")

# 3. Decision Tree

Write code for learning decision tree below. Take as an argument a hyperparameter on what size node to stop splitting. Use a part of training set as validation set.

Write code for running in the cell after (You may be asked to demonstrate your code during the viva using this cell.)

In text cell after that report the following numbers you get by running appropriate code:

For all four data sets  report the best node size to stop splitting. Report the training and test zero-one error for those hyperparameters.

For datasets A and B, also illustrate the learned classifier. Do this in the last codeWrite cell for this question.

Important: Think about how you will represent a decision tree. (Possible soln: Store as a list of tuples containing node position, attribute to split, threshold, class to classifiy (if leaf node) )


In [6]:
# CodeWrite cell
# Write Decision tree classifier from scratch, 
# write only functions here (you may write extra functions here if you wish)

class Node:
    def __init__(self, feature_index, threshold, left, right, value):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        

def train_decision_tree(X, Y, num_nodes_stop=1, criterion='accuracy'):
    """
    Returns a decision tree trained on X and Y. 
    Stops splitting nodes when a node has hit a size of "num_nodes_stop" or lower.
    Split criterion can be either 'accuracy' or 'entropy'.
    Returns a tree (In whatever format that you find appropriate)
    """

    def entropy(y):
        unique, counts = np.unique(y, return_counts=True)
        p = counts / counts.sum()
        return -np.sum(p * np.log2(p))

    def accuracy(y):
        unique, counts = np.unique(y, return_counts=True)
        return counts.max() / counts.sum()

    def split(X, Y, feature_index, threshold):
        left_indices = X[:, feature_index] < threshold
        right_indices = X[:, feature_index] >= threshold
        return X[left_indices], X[right_indices], Y[left_indices], Y[right_indices]

    def sort_by_X(X, Y):
        sorted_indices = np.argsort(X)
        return X[sorted_indices], Y[sorted_indices]

    def find_best_split(X, Y):
        X, Y = sort_by_X(X, Y)
        best_feature_index = None
        best_threshold = None
        best_gain = 0
        if criterion == 'accuracy':
            best_criterion = accuracy(Y)
        elif criterion == 'entropy':
            best_criterion = entropy(Y)
        
        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            X_left, X_right, Y_left, Y_right = split(X, Y, feature_index, thresholds[0])
            
            for threshold in thresholds:
                while X_right.shape[0] > 0 and X_right[0, feature_index] < threshold:
                    X_left = np.concatenate((X_left, X_right[0:1, :]), axis=0)
                    Y_left = np.concatenate((Y_left, Y_right[0:1]), axis=0)
                    X_right = X_right[1:, :]
                    Y_right = Y_right[1:]

                if criterion == 'accuracy':
                    gain = accuracy(Y_left) + accuracy(Y_right) - best_criterion
                elif criterion == 'entropy':
                    gain = entropy(Y_left) + entropy(Y_right) - best_criterion
                
                if gain > best_gain:
                    best_feature_index = feature_index
                    best_threshold = threshold
                    best_gain = gain
            
        return best_feature_index, best_threshold

    def build_tree(X, Y):
        if len(np.unique(Y)) == 1 or len(Y) <= num_nodes_stop:
            return Node(None, None, None, None, np.bincount(Y).argmax())
        feature_index, threshold = find_best_split(X, Y)
        X_left, X_right, Y_left, Y_right = split(X, Y, feature_index, threshold)
        left = build_tree(X_left, Y_left)
        right = build_tree(X_right, Y_right)
        return Node(feature_index, threshold, left, right, None)

    return build_tree(X, Y)

def eval_decision_tree(tree, test_X):
    """ 
    Takes in a tree, and a bunch of instances X and 
    returns the tree predicted values at those instances.
    """
    def predict(node, x):
        if node.value is not None:
            return node.value
        if x[node.feature_index] < node.threshold:
            return predict(node.left, x)
        else:
            return predict(node.right, x)

    return np.array([predict(tree, x) for x in test_X])





In [7]:
# CodeWrite cell
# Write code here for doing validation to find the best hyperparameters (i.e. num_nodes_stop)
# Also Generate the numbers that you report below. 
# Repeat with criterion set to entropy also.

def accuracy_score(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

def get_best_params_decision_tree(file):
    data = np.load("./dataset/"+file)
    x_train = data['arr_0']
    y_train = data['arr_1']
    x_test = data['arr_2']
    y_test = data['arr_3']

    best_params = {}
    for criterion in ['accuracy', 'entropy']:
        best_params[criterion] = {}
        for num_nodes_stop in range(1, 41):
            tree = train_decision_tree(x_train, y_train, num_nodes_stop, criterion)
            y_pred = eval_decision_tree(tree, x_test)
            best_params[criterion][num_nodes_stop] = accuracy_score(y_test, y_pred)
    return best_params

best_params = {}
best_params['dataset_A.npz'] = get_best_params_decision_tree("dataset_A.npz")
best_params['dataset_B.npz'] = get_best_params_decision_tree("dataset_B.npz")
print(best_params)

ValueError: zero-size array to reduction operation maximum which has no identity

TextWrite cell: Give your observations and the list of hyperparameter choices and train zero-one error  and test zero-one error, for all 4 datasets (2 real world and 2 synthetic).  



In [ ]:
## Codewrite cell: Generate plots of learned decision tree classifier on dataset_A and datasset_B.
# Plots should give both the learned classifier and the train data. 
# Plots only required for the accuracy criterion.
# Similar to  Bishop Figure 4.5 (with just two classes here.)
# Total number of plots = 2 




# 4 Random Forest classifier

Write code for learning RandomForests below. Fix the following hyper parameters: (Fraction of data to learn tree=0.5, Fraction of number of features chosen in each node=0.5, num_nodes_stop=1).  Choose the number of trees to add in the forest by using a validation set. You may use a slightly modified version of the decision tree code you had written earlier.

Write code for running in the cell after the nest. (You may be asked to demonstrate your code during the viva using this cell.) 

In text cell after that report the following numbers you get by running appropriate code:

For all 4 data sets (A,B,C,D)  report the best number of trees found. Report the training and test zero-one error for those hyperparameters.

For the synthetic classification datasets (datasets A and B) in 2-dimensions, also illustrate the learned classifier for each kernel setting. Do this in the last codeWrite cell for this question.

In [ ]:
# CodeWrite cell
# Write Random Forest classifier.

class Node:
    def __init__(self, feature_index, threshold, left, right):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right

def train_random_forest(X, Y, num_trees=10, num_nodes_stop=1, 
                        criterion='accuracy', a=0.5, b=0.5):
    """ 
    Returns a random forest trained on X and Y. 
    Trains num_trees.
    Stops splitting nodes in each tree when a node has hit a size of "num_nodes_stop" or lower.
    Split criterion can be either 'accuracy' or 'entropy'.
    Fraction of data used per tree = a
    Fraction of features used in each node = b
    Returns a random forest (In whatever format that you find appropriate)
    """
    def entropy(y):
        unique, counts = np.unique(y, return_counts=True)
        p = counts / counts.sum()
        return -np.sum(p * np.log2(p))

    def accuracy(y):
        unique, counts = np.unique(y, return_counts=True)
        return counts.max() / counts.sum()

    def split(X, Y, feature_index, threshold):
        left_indices = X[:, feature_index] < threshold
        right_indices = X[:, feature_index] >= threshold
        return X[left_indices], X[right_indices], Y[left_indices], Y[right_indices]

    def find_best_split(X, Y):
        best_feature_index = None
        best_threshold = None
        best_gain = 0
        if criterion == 'accuracy':
            best_criterion = accuracy(Y)
        elif criterion == 'entropy':
            best_criterion = entropy(Y)
        for feature_index in range(X.shape[1]):
            thresholds = np.unique(X[:, feature_index])
            for threshold in thresholds:
                X_left, X_right, Y_left, Y_right = split(X, Y, feature_index, threshold)
                if criterion == 'accuracy':
                    gain = accuracy(Y_left) + accuracy(Y_right) - best_criterion
                elif criterion == 'entropy':
                    gain = entropy(Y_left) + entropy(Y_right) - best_criterion
                if gain > best_gain:
                    best_feature_index = feature_index
                    best_threshold = threshold
                    best_gain = gain
        return best_feature_index, best_threshold
    
    def sample_data(X, Y, a):
        n = len(Y)
        indices = np.random.choice(n, int(n * a), replace=False)
        return X[indices], Y[indices]

    def sample_features(X, b):
        n = X.shape[1]
        indices = np.random.choice(n, int(n * b), replace=False)
        return X[:, indices]

    def build_tree(X, Y):
        if len(np.unique(Y)) == 1 or len(Y) <= num_nodes_stop:
            return Node(None, None, None, None)
        feature_index, threshold = find_best_split(X, Y)
        X_left, X_right, Y_left, Y_right = split(X, Y, feature_index, threshold)
        left = build_tree(X_left, Y_left)
        right = build_tree(X_right, Y_right)
        return Node(feature_index, threshold, left, right)

    forest = []
    for _ in range(num_trees):
        X_sample, Y_sample = sample_data(X, Y, a)
        X_sample = sample_features(X_sample, b)
        forest.append(build_tree(X_sample, Y_sample))
    return forest
    

def eval_random_forest(random_forest, test_X):
    """ 
    Takes in a  random forest object (hhowever you want to store it), and a bunch of instances X and 
    returns the tree predicted values at those instances.
    """
    def predict(node, x):
        if node.feature_index is None:
            return np.bincount(node.Y).argmax()
        if x[node.feature_index] < node.threshold:
            return predict(node.left, x)
        else:
            return predict(node.right, x)

    predictions = []
    for tree in random_forest:
        predictions.append(np.array([predict(tree, x) for x in test_X]))
    return np.array(predictions).mean(axis=0).round().astype(int)




In [ ]:
# CodeWrite cell
# Write code for choosing the best hyperparameters (num_trees, num_nodes_stop)
# Write code here for generating the numbers that you report below.
# Repeat above for criterion set to entropy also.



TextWrite cell: Give your observations and the list of hyperparameter choices and train zero-one error  and test zero-one error, for all 4 datasets (2 real world and 2 synthetic).  


In [3]:
## Codewrite cell: Generate plots of learned Random Forest classifier on dataset_A and datasset_B.
# Plots should give both the learned classifier and the train data. 
# Plots required only for the accuracy criterion.
# Similar to  Bishop Figure 4.5 (with just two classes here.)
# Total number of plots = 2 


# 5 AdaBoost

Write code for learning using AdaBoost below. Use 3 different weak learners below. (You may reuse code written above)

1. 1 node decision tree 
2. Decision tree of fixed depth = 3 (Root, child, grand child)
3. Decision tree of fixed depth = 7 (Root, child, grand child, ..., great^4 grand child)

Run for 50 iterations. You may use the accuracy split criterion for all the three weak learners.

Write code for running in the next cell. (You may be asked to demonstrate your code during the viva using this cell.) 

In text cell after that report the following numbers you get by running appropriate code:

For all 4 data sets (A,B,C,D)  plot the train and test accuracy vs iterations. A total of 12 plots is expected. 4 datasets * 3 weak learners. Each plot should contain two curves, train and test error.  

For the synthetic classification datasets (datasets A and B) in 2-dimensions, also illustrate the learned classifier for each weak learner setting. A total of 6 contourf style plots are expected here. Do this in the last codeWrite cell for this question.

Summarise your observations in the last textwrite cell.

In [4]:
# Codewrite cell
# Write code to run here (no plotting)

def ada_boost(X, Y, num_boosting_iters=50, num_nodes_stop=1, criterion='accuracy'):
    """ 
    Returns a list of weak classifiers (i.e. decision trees) trained on X and Y.
    Trains num_boosting_iters weak classifiers.
    Stops splitting nodes in each tree when a node has hit a size of "num_nodes_stop" or lower.
    Split criterion can be either 'accuracy' or 'entropy'.
    Returns a list of weak classifiers (In whatever format that you find appropriate)
    """


In [5]:
# Codewrite cell 
# Plots for iteration vs error here


In [6]:
# Codewrite cell 
# Plots for illustrating the classifier here


Textwrite cell: